# Shell Commands

## Compound Commands

### Looping Constructs

#### Until

In [1]:
%%bash

i=1
until [ $i -ge 5 ]; 
do
    echo $i
    ((i++))
done

1
2
3
4


#### While

In [2]:
%%bash

i=1
while [ $i -le 5 ]
do
    echo $i
    ((i++))
done

1
2
3
4
5


#### For

In [3]:
%%bash

for NUMBER in 1 2 3; do
    echo $NUMBER
done

1
2
3


With range and step.

In [4]:
%%bash

for NUMBER in {1..10..2}; do
    echo $NUMBER
done

1
3
5
7
9


With an array.

In [5]:
%%bash

NUMBERS=(1 2 3 4 5)
for NUMBER in ${NUMBERS[@]}; do
    echo $NUMBER
done

1
2
3
4
5


Three expressions

In [6]:
%%bash

for (( c=10; c<=20; c++ ))
do
    if [ "$c" -le 12 ]; then
        continue
    elif [ "$c" -gt "15" ]; then
        break
    fi
    echo $c
done

13
14
15


### Conditional Constructs

#### [] vs [[]]

`[]` is a traditional shell test command. `[[]]` is a test command that also supports regular expressions.

#### If

In [7]:
%%bash

NUM=5
if [[ "$NUM" -eq 5 ]]; then
    echo true
else
    echo false
fi

true


#### Case

Basic.

In [8]:
%%bash

NAME=john
case $NAME in
    john | John | JOHN)echo "match";;
    jan | "Jan Jan") echo "match";;
    # Default
    *) echo "Not found";;
esac

match


Execute next.

In [9]:
%%bash

NAME=a
case $NAME in
  a) echo "match";&
  b) echo "match";;
  c) echo "match";;
  c) echo "match";;
esac

match
match


Test next.

In [10]:
%%bash

NAME=a
case $NAME in
  a) echo "match";;&
  b) echo "match";;
  c) echo "match";;
  c) echo "match";;
esac

match


#### Select

One of the options.

In [11]:
%%bash

OPTIONS=( a b c )
select CHOISE in ${OPTIONS[@]}
do
    echo "Selection: $CHOISE"
    break;
done

1) a
2) b
3) c
#? 

Files from current directory.

In [12]:
%%bash

select fname in *;
do
    echo Selection $fname \($REPLY\)
    break;
done

1) README.md
2) bash.ipynb
3) files
4) function.sh
5) test.sh
#? 

#### Arithmetic expressions

Returns 0 if result is grether than 0.

In [13]:
%%bash

if ((5)); then
  echo True
else
  echo False
fi

True


Returns 1 if result is equal 0

In [14]:
%%bash

if ((0)); then
  echo False
else
  echo True
fi

True


#### Conditional expressions

Regular expression.

In [15]:
%%bash

name=file.txt
pattern='\.txt'
[[ $name =~ $pattern ]] && echo match

match


Invalid regular expression. This doesn't work because ' is a part of the pattern.

In [16]:
%%bash

name=file.txt
# This doesn't work because ' is a part of the pattern
[[ $name =~ '\.txt' ]] || echo "doesn't match"

doesn't match


Boolean operation.

In [17]:
%%bash

name=file.txt
pattern1='\.txt'
pattern2='file\.'
[[ ($name =~ $pattern1) && ($name =~ $pattern2) ]] && echo match

match


Bracket globbing.

In [18]:
%%bash

name=file.txt
pattern='\.txt'
[[ $name = [fgr]ile.txt ]] && echo match

match


### Grouping Commands

Grouping without side effects using `()`.

In [19]:
%%bash

counter=1
( ((counter++)); echo "local: $counter" )
echo "global: $counter"

local: 2
global: 1


Grouping with side effects using `{}`.

In [20]:
%%bash

counter=1
{ ((counter++)); echo "local: $counter"; }
echo "global: $counter"

local: 2
global: 2


## Coprocesses

Replaces `a` to `b`. `COPROC` is a default variable name.

In [21]:
%%bash

coproc { tr a b; }
echo "PID: $COPROC_PID"
echo aaa >&"${COPROC[1]}" # standard input [1]
exec {COPROC[1]}>&-
cat <&"${COPROC[0]}" # standard output [0]

PID: 88
bbb


In [22]:
%%bash

coproc name { tr [:lower:] [:upper:]; }
echo "PID: $name_PID"
echo aAa >&"${name[1]}"
exec {name[1]}>&-
cat <&"${name[0]}"

PID: 92
AAA


## GNU Parallel

TODO

# Shell Functions

Without reserved name `function`.

In [23]:
%%bash

f1() (
  echo "Number of arguments: $#"
)
f1 a b c

Number of arguments: 3


With reserved word `function`.

In [24]:
%%bash

function f {
  echo "Third argument: $3";
}
f a b c

Third argument: c


Remove function.

In [25]:
%%bash

function f {
  echo "Third argument: $3";
}
unset -f f && echo removed

removed


Local variables.

In [26]:
%%bash

loc='initial value'
f() (
  local loc='local value'
  glo=global
  echo $loc
)
f
echo $loc
echo $glo

local value
initial value



Print function declaration.

In [27]:
%%bash

f() (
    echo "Hello!"
)
declare -f f

f () 
{ 
    ( echo "Hello!" )
}


Recursive function call. `FUNCNEST` defines a maximum function nesting level.

In [28]:
%%bash

counter=5
f () (
  echo Call recursivelly: $counter
  if [[ "$counter" -gt 1 ]]; then
    ((counter--))
    f
  fi
)
f

Call recursivelly: 5
Call recursivelly: 4
Call recursivelly: 3
Call recursivelly: 2
Call recursivelly: 1


Export function.

`function.sh` code:
```
#!/bin/bash

f f r e r e
```

In [29]:
%%bash

f() (
    echo "Number of arguments: $#"
)
export -f f
./function.sh

Number of arguments: 5


# Shell Parameters

## Positional Parameters

Declare shell parameter.

In [30]:
%%bash

declare var=a
echo parameter value is $var

parameter value is a


Appending to string.

In [31]:
%%bash

num=666
num+=34
echo String $num

String 66634


Append to integer.

In [32]:
%%bash

declare -i num=666
num+=34
echo Integer $num

Integer 700


Declare reference to variable.

In [33]:
%%bash

declare -n ref=num
num+=13
ref+=10
echo original value: $num reference: $ref

original value: 1310 reference: 1310


In [34]:
%%bash

f() (
  shift 1
  echo parameters: $# first: $1 second: $2
)
f a b c

parameters: 2 first: b second: c


Print all shell variables and functions.
```
#!/bin/bash
set
```

Set positional parameters.

In [35]:
%%bash

f() (
  set x y z
  echo parameters: $*
)
f a b c

parameters: x y z


### Special Parameters

Difference between `$*` and `$@`.
```
# $*	$1 $2 $3...
# $@	$1 $2 $3...
# "$*"	"$1 $2 $3..."
# "$@"	"$1" "$2" "$3"...
```

In [36]:
%%bash

f() (
  echo parameters: $*
  echo parameters: $@
  echo number of parameters: $#
  echo last exit code: $?
  echo current options flags: $-
  echo process id: $$
  cat&
  echo last background process: $!
  echo name of the shell: $0
  echo startup: $_
)
f aaa bbb ccc

parameters: aaa bbb ccc
parameters: aaa bbb ccc
number of parameters: 3
last exit code: 0
current options flags: hB
process id: 119
last background process: 121
name of the shell: bash
startup: bash


# Shell Expansions

The order of expansions is:

- brace expansion; 
- tilde expansion, parameter and variable expansion, arithmetic expansion, and command substitution (done in a left-to-right fashion);
- word splitting; 
- filename expansion;

### Brace Expansion

In [37]:
%%bash

echo 1{a,b,c}
echo {01..10..2}

1a 1b 1c
01 03 05 07 09


### Tilde expansion

In [38]:
%%bash

echo '$HOME:' ~
echo '$PWD:' ~+
echo '$OLDPWD:' ~-



$HOME: /home/sage
$PWD: /home/sage/bash
$OLDPWD: ~-


Directory stack. Commands `pushd` and `popd`.

In [39]:
%%bash

echo 'add $HOME to the directory stack'
pushd ~
echo print last and previous directories: ~+0 ~+1

add $HOME to the directory stack
~ ~/bash
print last and previous directories: /home/sage /home/sage/bash


### Shell Parameter Expansion

In [40]:
%%bash

echo basic: ${HOME}
echo without optional braces: $HOME

basic: /home/sage
without optional braces: /home/sage


#### Indirect expansion

In [41]:
%%bash

var=value
varName="var"
declare -n varRef=var
echo expands by variable name and returns value: ${!varName}
echo expands reference and returns variable name: ${!varRef}

echo "expands variables starting with 'va':" ${!va*}
echo "expands variables starting with 'va':" ${!va@}

array=(a b c)
echo expand array indices: ${!array[*]}
echo expand array indices: ${!array[@]}


expands by variable name and returns value: value
expands reference and returns variable name: var
expands variables starting with 'va': var varName varRef
expands variables starting with 'va': var varName varRef
expand array indices: 0 1 2
expand array indices: 0 1 2


Default value.

In [42]:
%%bash

var=value
unsetVar="unsetValue"
unset unsetVar
nullVar=
defVar=defaultValue

echo ignore default value: ${var:-$defVar}
echo use default instead of the null: ${nullVar:-$defVar}
echo use default instead of the unset: ${unsetVar:-otherDefaultValue}

ignore default value: value
use default instead of the null: defaultValue
use default instead of the unset: otherDefaultValue


Conditional default value.

In [43]:
%%bash

var=value
unsetVar="unsetValue"
unset unsetVar
nullVar=
defVar=defaultValue

echo ignore setting a default value: ${var:=$defVar}
echo set default value if null: ${nullVar:=$defVar} ${nullVar}
echo set default value if unset: ${unsetVar:=$defVar} ${unsetVar}

ignore setting a default value: value
set default value if null: defaultValue defaultValue
set default value if unset: defaultValue defaultValue


Invalid usage.

In [44]:
%%bash

var=value

echo set default to the positional parameter: ${1:=$var} $1

bash: line 4: $1: cannot assign in this way


Validation.

In [45]:
%%bash

var=value
unsetVar="unsetValue"
unset unsetVar
nullVar=
defVar=defaultValue

echo ignore validation: ${var:?'Invalid var'}
echo validate null: ${nullVar:?'Variable is Null'}
echo validate unset: ${unsetVar:?'Variable is unset'}

ignore validation: value


bash: line 9: nullVar: Variable is Null


Set default if variable has a value.

In [46]:
%%bash

var=value
unsetVar="unsetValue"
unset unsetVar
nullVar=
defVar=defaultValue

echo if variable has value then set default: ${var:+defVar}
echo do nothing if null: ${nullVar:+defVar}
echo do nothing if unset: ${unsetVar:+defVar}

if variable has value then set default: defVar
do nothing if null:
do nothing if unset:


#### Substring expansion

In [47]:
%%bash

var=value

echo first char: ${var:1:1}
echo substring: ${var:1:-1}
echo last three chars: ${var: -3}

array=(a b c d e)
echo second and third array element: ${array[@]:1:2}
echo second and third array element: ${array[*]:1:2}

f() (
    echo second and third parameters: ${@:2:2}
    echo second and third parameters: ${*:2:2}
    echo last parameter: ${*: -1}
)
f a b c d e

first char: a
substring: alu
last three chars: lue
second and third array element: b c
second and third array element: b c
second and third parameters: b c
second and third parameters: b c
last parameter: e


#### Length Expansion

In [48]:
%%bash

var=value
array=(a b c d e)

echo string length: ${#var}
echo array length: ${#array[*]}
echo array length: ${#array[@]}

array=(aaa bb c)
echo array elements length: ${#array[-1]} ${#array[-2]} ${#array[-3]}
echo array elements length: ${#array[0]} ${#array[1]} ${#array[2]}

string length: 5
array length: 5
array length: 5
array elements length: 1 2 3
array elements length: 3 2 1


#### Pattern expansion

From begin.

In [49]:
%%bash

var='home/user/app.log'
array=('home/user/app.log' 'home/user/srv.log' 'home/user/svc.log')

echo match shortest: ${var#*/}
echo match shortest: ${array[@]#*/}
echo match longest: ${var##*/}
echo match longest: ${array[@]##*/}

match shortest: user/app.log
match shortest: user/app.log user/srv.log user/svc.log
match longest: app.log
match longest: app.log srv.log svc.log


From the end.

In [50]:
%%bash

var='home/user/app.log'
array=('home/user/app.log' 'home/user/srv.log' 'home/user/svc.log')

echo match shortest: ${var%/*}
echo match shortest: ${array[@]%/*}
echo match longest: ${var%%/*}
echo match longest: ${array[@]%%/*}

match shortest: home/user
match shortest: home/user home/user home/user
match longest: home
match longest: home home home


Replace.

In [51]:
%%bash

var='home/user/app.log'

echo replace longest: ${var/user/john}
echo replace longest from the start: ${var/#home/etc}
echo replace longest from the end: ${var/%log/txt}

replace longest: home/john/app.log
replace longest from the start: etc/user/app.log
replace longest from the end: home/user/app.txt


Case expansion.

In [52]:
%%bash

var='home/user/app.log'
echo first char to upper case: ${var^} same as ${var^?}
echo first char to upper case: ${var^s}
echo all matches to upper case: ${var^^s}

var=USeruSer
echo first char to lower case: ${var,U}
echo all matches to lower case: ${var,,S}

first char to upper case: Home/user/app.log same as Home/user/app.log
first char to upper case: home/user/app.log
all matches to upper case: home/uSer/app.log
first char to lower case: uSeruSer
all matches to lower case: Useruser


Transformation expansion.

In [53]:
%%bash

var='\"value'
echo input: ${var@Q}
echo escape: ${var@E} same as $'\"value'

var='\"value\h'
echo prompt: ${var@P}

var='newValue'
echo evaluate: ${var@A} ${var}

bash: line 3: ${var@Q}: bad substitution
bash: line 4: ${var@E}: bad substitution
bash: line 7: ${var@P}: bad substitution
bash: line 10: ${var@A}: bad substitution


 Information expansion.

In [54]:
%%bash

declare -i var=1
echo expand declaration: ${var@A}
echo expand flags: ${var@a}

bash: line 3: ${var@A}: bad substitution
bash: line 4: ${var@a}: bad substitution


## Command Substitution

Executes command in the subshell.

In [55]:
%%bash

msg=Hello!
echo $(echo "$msg") same as `echo $msg`

Hello! same as Hello!


Read from file

In [56]:
%%bash

echo read from file \"$(<'files/data')\" same as \"$(cat 'files/data')\" but faster

read from file "John Dave Jim" same as "John Dave Jim" but faster


## Arithmetic Expansion

Evaluates arithmetic expressions.

In [57]:
%%bash

expression='21 * 2'
echo $(($expression))

42


## Process Substitution

Passing filename. In this example cat gets filename and read it. 

In [58]:
%%bash
cat <(date)

Tue Aug 11 14:14:55 UTC 2020


In this example echo doesn't read filename it just prints it.

In [59]:
%%bash
echo date: <(date)

date: /dev/fd/63


date: write error: Broken pipe


It needs to be piped because echo doesn't read stdin, but it reads parameters.

In [60]:
%%bash
echo date: < <(date)

date:


date: write error: Broken pipe


In [61]:
%%bash

comm -3 <(sort 'files/file1' | uniq) <(sort 'files/file2' | uniq)

a
b
	cx
e
	h


Piping stderr to other process.

In [62]:
%%bash

name="files/invalid"
pattern="cat: ${name/\//\\/}: "
(cat $name >/dev/null) 2> >(sed "s/${pattern}//")

No such file or directory


## Word Splitting

`$IFS` is a delimiter that is using for word splitting.

## Filename Expansion

After executing word splitting Bash performs scanning for `*`, `?` and `[`. Then replaces them with filenames.

#### *

In [63]:
%%bash

ls files/*

files/2.log
files/22.log
files/data
files/file1
files/file2
files/file3
files/tmp


`/**` matches only directories and subdirectories.

In [64]:
%%bash

ls ./**/*

./files/2.log
./files/22.log
./files/data
./files/file1
./files/file2
./files/file3
./files/tmp


#### ?

In [65]:
%%bash

ls files/file?

files/file1
files/file2
files/file3


#### []

In [66]:
%%bash

ls files/file[13]

files/file1
files/file3


In [67]:
%%bash

ls files/file[1-3]

files/file1
files/file2
files/file3


### Extended pattern matching

To enable extended pattern matching it needs to set option `extglob`.
```
shopt -s extglob
```

In [68]:
%%bash

shopt -s extglob

# zero or one
ls files?(2)
echo
# zero or more
ls files*(2)
echo
# one or more
ls files/+(2).log
echo
# one
ls files/@(2).log
echo
# anything except one
ls files/!(2).log

2.log
22.log
data
file1
file2
file3
tmp

2.log
22.log
data
file1
file2
file3
tmp

files/2.log
files/22.log

files/2.log

files/22.log


# Redirection

#### Order of redirections is significant

`stdout` and `stderr` are redirected to temp file.

In [69]:
%%bash

t=$(tempfile) || exit
trap "rm -f -- '$t'" EXIT

ls invalid > $t 2>&1

`stderr` aren't redirected, because `stderr` makes copy of `stdout` before it was redirected. And you can see error message.

In [70]:
%%bash

t=$(tempfile) || exit
trap "rm -f -- '$t'" EXIT

ls invalid 2>&1 > $t

ls: cannot access 'invalid': No such file or directory


#### Redirecting input

In [71]:
%%bash

cat <files/data

John
Dave
Jim

Read first line into `var` variable.

In [72]:
%%bash

read -r var <files/data
echo $var

John


#### Redirecting Output

In [73]:
%%bash

echo "line" >files/tmp
cat files/tmp
rm files/tmp

line


If files are protected from the overwriting you have to use `>|`.

In [74]:
%%bash

# Protect files from overwriting.
set -o noclobber

touch files/tmp

echo "line" >|files/tmp
cat files/tmp
rm files/tmp

line


#### Appending

In [75]:
%%bash

echo "line1" >files/tmp
echo "line2" >>files/tmp
cat files/tmp
rm files/tmp

line1
line2


#### Redirecting Standard Output and Standard Error

`&>word` and `>&word` are same and are equiualent to `>word 2>&1`

#### Appending Standard Output and Standard Error

`&>>word` is equiualent to `>>word 2>&1`

#### Here Documents

`<<` reads lines until it spot delimiter. Use `<<-` to streap tabs.

In [76]:
%%bash

file=files/tmp

cat <<DELIMITER > $file
  one
  two
  three
DELIMITER

cat $file
rm $file

  one
  two
  three


#### Here Strings

In [77]:
%%bash

read first second <<< "Hello world"
echo $first, $second

Hello, world


#### Duplicating File Descriptors

In [78]:
%%bash

file=files/tmp

# copy
exec 3>$file
echo line >&3
# close
exec 3>&-
cat $file
rm $file

line


#### Moving File Descriptor

In [79]:
%%bash

3>&4-
echo same as
3>&4 4>&-

same as


bash: line 2: 4: Bad file descriptor
bash: line 4: 4: Bad file descriptor


#### Opening File Descriptors for Reading and Writing

Opens file for read write and create new if it doesn't exist.

In [80]:
%%bash

file=files/tmp
# don't fail if file doesn't exist
cat <>$file